# Fullstack GPT: #6.0 ~ #6.10

- Tasks:
- [x] Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- [x] 체인을 수동으로 구현해야 합니다.
- [x] 체인에 ConversationBufferMemory를 부여합니다.
- [x] 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
- [x] 체인에 다음 질문을 합니다:
    - [x] Is Aaronson guilty?
    - [x] What message did he write in the table?
    - [x] Who is Julia?
- 다음과 같은 절차대로 구현하면 챌린지를 해결할 수 있습니다.
    - [x] (1) 문서 로드하기 : [TextLoader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/) 등 을 사용해서 파일에서 텍스트를 읽어옵니다.
    - [x] (2) 문서 쪼개기 : [CharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/character_text_splitter/) 등 을 사용해서 문서를 작은 문서 조각들로 나눕니다.
    - [x] (3) 임베딩 생성 및 캐시 : OpenAIEmbeddings, [CacheBackedEmbeddings](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/caching_embeddings/) 등 을 사용해 문서 조각들을 임베딩하고 임베딩을 저장합니다.
    - [x] (4) 벡터 스토어 생성 : [FAISS](https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/) 등 을 사용해서 임베딩된 문서들을 저장하고 검색할 수 있는 데이터베이스를 만듭니다.
    - [x] (5) 대화 메모리와 질문 처리 : ConversationBufferMemory를 사용해 대화 기록을 관리합니다.
    - [x] (6) 체인 연결 : 앞에서 구현한 컴포넌트들을 적절하게 체인으로 연결합니다.

In [10]:
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def embedder():
    FILE_PATH = "../files/document.txt"

    loader = TextLoader(FILE_PATH)
    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n\n",
        chunk_size=600,
        chunk_overlap=100,
    )
    docs = loader.load_and_split(splitter)

    cache_dir = LocalFileStore("../files/.cache/")
    embeddings = OpenAIEmbeddings()
    cached_embbedings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

    vectorstore = FAISS.from_documents(docs, cached_embbedings)
    retriever = vectorstore.as_retriever()
    return retriever

def load_memory(_):
    return memory.load_memory_variables({})["history"]

def invoke_chain(query):
    response = chain.invoke(query)
    memory.save_context({"input":query}, {"output": response.content})

set_llm_cache(InMemoryCache())

memory = ConversationBufferMemory(return_messages=True)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)
retriever = embedder()
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Answer human question according to following context and conversation history. If you don't know don't make it up and just say so.\n\n
    {context}
    """),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = {"context":retriever, "history":RunnableLambda(load_memory), "question":RunnablePassthrough()} | prompt | llm
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

load_memory("input")

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content="According to the context provided, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they were charged with. However, he also acknowledges that he has never seen the photograph that disproved their guilt and that it had never existed, implying that their guilt is a fabrication of the Party. Therefore, while the Party claims Aaronson is guilty, Winston's realization suggests that the truth is more complex and likely indicates that Aaronson is not actually guilty."),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='Winston traced "2+2=5" in the dust on the table.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content="Julia is a character in the story who is Winston's love interest. She is a fellow Party member who initially appears to conform to the Party's rules but secretly rebels against its oppressive regime. Julia and Winston share a r